This is the first try on implementing a LSTM on this project.

In [19]:
# Importación de librerías para manejo de datos, visualización y deep learning
# Incluye librerías para procesamiento, modelado y reproducibilidad de resultados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf


# Preprocesamiento y modelo LSTM
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# Funciones personalizadas del proyecto
from comparation2 import comparation2
from Resumir_Datasets import resumir_datasets
from unionDatasets import union_Datasets


# Semilla fija para reproducibilidad de resultados
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

In [20]:
# Ejecuta la función que resume los datasets originales y guarda los archivos resumidos en la carpeta datasets_resumidos/
# Entrada: Archivos como 2013A.xlsx, 2013B.xlsx, etc. en la carpeta datasets_originales/
# Salida: Archivos como resumen_cupos_2013A.xlsx, resumen_cupos_2013B.xlsx, etc. en la carpeta datasets_resumidos/
resumir_datasets()


Archivo generado: datasets_resumidos\resumen_cupos_2013A.xlsx
Funcion de resumir los datasets ejecutada con exito
Archivo generado: datasets_resumidos\resumen_cupos_2013B.xlsx
Funcion de resumir los datasets ejecutada con exito
Archivo generado: datasets_resumidos\resumen_cupos_2014A.xlsx
Funcion de resumir los datasets ejecutada con exito
Archivo generado: datasets_resumidos\resumen_cupos_2014B.xlsx
Funcion de resumir los datasets ejecutada con exito
Archivo generado: datasets_resumidos\resumen_cupos_2015A.xlsx
Funcion de resumir los datasets ejecutada con exito
Archivo generado: datasets_resumidos\resumen_cupos_2015B.xlsx
Funcion de resumir los datasets ejecutada con exito
Archivo generado: datasets_resumidos\resumen_cupos_2016A.xlsx
Funcion de resumir los datasets ejecutada con exito
Archivo generado: datasets_resumidos\resumen_cupos_2016B.xlsx
Funcion de resumir los datasets ejecutada con exito
Archivo generado: datasets_resumidos\resumen_cupos_2017A.xlsx
Funcion de resumir los dat

In [21]:
# Une todos los archivos resumidos en un solo archivo CSV llamado 'oferta_academica_unificada.csv'
# Entrada: Archivos resumen_cupos_*.xlsx en datasets_resumidos/
# Salida: Un solo archivo oferta_academica_unificada.csv con todas las materias y periodos juntos
union_Datasets()

Iniciando la funcion
Archivo guardado como 'oferta_academica_unificada.csv
Funcion de union de datasets ejecutada con exito


In [22]:
# Carga el archivo unificado con todas las materias y periodos
df = pd.read_csv("oferta_academica_unificada.csv")

# Elimina filas que no tienen datos clave (Materia, Total_Cupos, semestre_numerico)
df = df.dropna(subset=['Materia', 'Total_Cupos', 'semestre_numerico'])

# Codifica la columna 'Materia' a números para que pueda ser usada por el modelo, como un diccionario
le = LabelEncoder()
df['materia_codificada'] = le.fit_transform(df['Materia'])

# Guarda el mapeo de códigos a nombres de materia para referencia futura
diccionario_materias = dict(zip(df['materia_codificada'], df['Materia']))

# Crea una columna con los cupos efectivamente usados (Total_Cupos - Residuos_Cupos)
df['Cupos_Usados'] = df['Total_Cupos'] - df['Residuos_Cupos'].fillna(0)

# Define las columnas que serán escaladas (normalizadas entre 0 y 1)
escalar = ['Total_Secciones', 'semestre_numerico', 'Cupos_Usados', 'Residuos_Cupos']
scaler = MinMaxScaler()
df[escalar] = scaler.fit_transform(df[escalar])

# Define las columnas de entrada (features) y la columna objetivo (target)
features = ['materia_codificada', 'Total_Secciones', 'semestre_numerico', 'Residuos_Cupos', 'Cupos_Usados']
target = 'Cupos_Usados'

# Ejemplo de entrada:
# | Materia         | Total_Cupos | Residuos_Cupos | semestre_numerico | ... |
# |-----------------|-------------|----------------|-------------------|-----|
# | PROGRAMACION    | 100         | 10             | 1                 | ... |
#
# Ejemplo de salida (después de procesamiento):
# | materia_codificada | Total_Secciones | semestre_numerico | Residuos_Cupos | Cupos_Usados | ... |
# |--------------------|-----------------|-------------------|----------------|--------------|-----|
# | 0                  | 0.2             | 0.0               | 0.1            | 0.9          | ... |

#   - Escalar los datos y codificar las etiquetas es fundamental para que el modelo LSTM pueda aprender patrones útiles y generalizar bien.
#   - Tener los datos entre 0 y 1 ayuda a que el entrenamiento sea más eficiente y estable.


In [23]:
"""
Convierte una matriz de datos en secuencias para alimentar a una red LSTM.
    data: np.array de tamaño (n, m) donde n=filas, m=features+target
    pasos: int, número de pasos de la secuencia (ventana temporal)
    Devuelve: X (secuencias de entrada), y (targets)
    Ejemplo de entrada:
        data = [[0, 0.2, 0.0, 0.1, 0.9, 0.9],   # fila 1
                [0, 0.3, 0.1, 0.2, 0.8, 0.8],   # fila 2
                ...]
        pasos = 6
    Ejemplo de salida:
        X.shape = (n_secuencias, 6, 5)
        y.shape = (n_secuencias,)
"""
def crear_secuencias(data, pasos):
    X, y = [], []
    for i in range(len(data) - pasos):
        X.append(data[i:i+pasos, :-1])
        y.append(data[i+pasos, -1])
    return np.array(X), np.array(y)

pasos = 6 # Número de periodos a considerar en cada secuencia
X_total, y_total = [], []

# Para cada materia, crea las secuencias de entrada y salida
for materia_id in df['materia_codificada'].unique():
    
    grupo = df[df['materia_codificada'] == materia_id].sort_values('semestre_numerico')
    #print(grupo)
    valores = grupo[features + [target]].values
    if len(valores) > pasos:
        X_seq, y_seq = crear_secuencias(valores, pasos)
        X_total.append(X_seq)
        y_total.append(y_seq)

# Une todas las secuencias de todas las materias en un solo arreglo
X = np.vstack(X_total)
y = np.concatenate(y_total)

# Ejemplo de salida:
# X.shape = (total_secuencias, pasos, features)
# y.shape = (total_secuencias,)
print("✅ Secuencias creadas:", X.shape)
    


✅ Secuencias creadas: (543, 6, 5)


In [24]:
# Construcción del modelo LSTM con los mejores hiperparámetros encontrados
# Estructura: 2 capas LSTM, 1 capa densa, 1 capa Dropout, 1 capa de salida

model = Sequential()
# First Layer
model.add(LSTM(96, return_sequences=True, input_shape=(pasos, len(features))))

# Second Layer
model.add(LSTM(64, return_sequences=False))

# 3rd Layer (Dense)
model.add(Dense(192, activation="relu")) # Toma las secuencias de la capa anterior y las transforma 

# 4th Layer (Dropout)
model.add(Dropout(0.2)) #apaga aleatoriamente el 20% de las neuronas durante el entrenamiento para que generalice mejor

# Final Output Layer
model.add(Dense(1))

# Entrenamiento del modelo
#model.compile(optimizer=Adam(learning_rate=0.1), loss='mse')
model.compile(optimizer='adam', loss='mse')
history = model.fit(X, y, epochs=100, verbose=1)



Epoch 1/100
17/17 [==============================] - 4s 11ms/step - loss: 0.0450
Epoch 2/100
17/17 [==============================] - 0s 11ms/step - loss: 0.0347
Epoch 3/100
17/17 [==============================] - 0s 12ms/step - loss: 0.0310
Epoch 4/100
17/17 [==============================] - 0s 12ms/step - loss: 0.0200
Epoch 5/100
17/17 [==============================] - 0s 11ms/step - loss: 0.0134
Epoch 6/100
17/17 [==============================] - 0s 13ms/step - loss: 0.0095
Epoch 7/100
17/17 [==============================] - 0s 12ms/step - loss: 0.0105
Epoch 8/100
17/17 [==============================] - 0s 12ms/step - loss: 0.0106
Epoch 9/100
17/17 [==============================] - 0s 12ms/step - loss: 0.0108
Epoch 10/100
17/17 [==============================] - 0s 12ms/step - loss: 0.0102
Epoch 11/100
17/17 [==============================] - 0s 12ms/step - loss: 0.0089
Epoch 12/100
17/17 [==============================] - 0s 12ms/step - loss: 0.0085
Epoch 13/100
17/17 [=====

In [25]:
# Predice el número de cupos usados para el próximo semestre de cada materia
# Para cada materia, toma la última secuencia y predice el siguiente valor
# El resultado se desescala para obtener el valor original

predicciones = {}

for materia_id in df['materia_codificada'].unique():
    materia_df = df[df['materia_codificada'] == materia_id].sort_values('semestre_numerico')
    valores = materia_df[features + [target]].values

    if len(valores) >= pasos:
        # Toma la última secuencia de la materia
        secuencia = valores[-pasos:, :-1].reshape(1, pasos, len(features))
        y_pred = model.predict(secuencia)[0][0]

        # Prepara la fila para desescalar el resultado
        ultima_fila_real = materia_df[escalar].values[-1].copy()
        ultima_fila_real[-1] = y_pred
        
        # Desescala el resultado a su valor original
        cupo_estimado = max(0, scaler.inverse_transform([ultima_fila_real])[0][-1])

        nombre_materia = diccionario_materias.get(materia_id, f"ID {materia_id}")
        predicciones[nombre_materia] = round(cupo_estimado)
# Ejemplo de salida:
# predicciones = {
#   "PROGRAMACION": 145,
#   "ESTRUCTURAS DE DATOS": 80,
#   ...
# }


1/1 [==============================] - 0s 20ms/step


In [26]:

# Guarda las predicciones en un archivo Excel para su análisis posterior

df_pred = pd.DataFrame(list(predicciones.items()), columns=["Materia", "Cupos_Estimados"])
df_pred.to_excel("predicciones_cupos_proximo_semestre.xlsx", index=False)
print("✅ Archivo guardado: predicciones_cupos_proximo_semestre.xlsx")



✅ Archivo guardado: predicciones_cupos_proximo_semestre.xlsx


In [27]:
# Compara las predicciones con los valores reales y muestra métricas de error

comparation2()

                                                                                  Materia  Total_Cupos  Cupos_Usados  Cupos_Estimados  Error_Absoluto  Desviacion_%
                                                                   ESTRUCTURAS DE DATOS I           40             1               13            12.0   1200.000000
                             SEMINARIO DE SOLUCION DE PROBLEMAS DE ESTRUCTURAS DE DATOS I           24             9               30            21.0    233.333333
                                                                  ESTRUCTURAS DE DATOS II           38             8                0             8.0   -100.000000
                                                                 INGENIERIA DE SOFTWARE I           44            43                4            39.0    -90.697674
                                                                               ALGORITMIA           40            26                9            17.0    -65.384615
                